In [ ]:
##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 开始使用 TensorBoard

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/tensorboard/get_started"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png"> 在 TensorFlow.org 上查看</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tensorboard/get_started.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tensorboard/get_started.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tensorboard/get_started.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a> </td>
</table>

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 设置日志级别为ERROR，以减少警告信息
# 禁用 Gemini 的底层库（gRPC 和 Abseil）在初始化日志警告
os.environ["GRPC_VERBOSITY"] = "ERROR"
os.environ["GLOG_minloglevel"] = "3"  # 0: INFO, 1: WARNING, 2: ERROR, 3: FATAL
os.environ["GLOG_minloglevel"] = "true"
import logging
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
!export TF_FORCE_GPU_ALLOW_GROWTH=true
from pathlib import Path

temp_dir = Path(".temp")
temp_dir.mkdir(parents=True, exist_ok=True)

在机器学习中，要改进模型的某些参数，您通常需要对其进行衡量。TensorBoard 是用于提供机器学习工作流期间所需测量和呈现的工具。它使您能够跟踪实验指标（例如损失和准确率），呈现模型计算图，将嵌入向量投影到较低维度的空间等。

本快速入门将展示如何快速使用 TensorBoard 。该网站上的其余指南提供了有关特定功能的更多详细信息，此处未包括其中的许多功能。 

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
import tensorflow as tf
import datetime

In [5]:
# Clear any logs from previous runs
!rm -rf {temp_dir}./logs/ 

在本例中使用 [MNIST](https://en.wikipedia.org/wiki/MNIST_database) 数据集。接下来编写一个函数对数据进行标准化，同时创建一个简单的Keras模型使图像分为10类。

In [7]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(name='layers_flatten'),
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
  ])

## 通过 Keras Model.fit() 使用 TensorBoard

当使用 Keras's [Model.fit()](https://tensorflow.google.cn/api_docs/python/tf/keras/models/Model#fit) 函数进行训练时, 添加 `tf.keras.callback.TensorBoard` 回调可确保创建和存储日志.另外，在每个时期启用 `histogram_freq=1` 的直方图计算功能（默认情况下处于关闭状态）

将日志放在带有时间戳的子目录中，以便轻松选择不同的训练运行。

In [9]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = temp_dir/"logs/fit"/datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5


I0000 00:00:1729772355.509210 3633479 service.cc:146] XLA service 0x7f8fa00077b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729772355.509244 3633479 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
I0000 00:00:1729772355.509248 3633479 service.cc:154]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
I0000 00:00:1729772357.575475 3633479 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - accuracy: 0.8949 - loss: 0.3614 - val_accuracy: 0.9697 - val_loss: 0.1054
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9688 - loss: 0.1003 - val_accuracy: 0.9737 - val_loss: 0.0842
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9780 - loss: 0.0698 - val_accuracy: 0.9773 - val_loss: 0.0686
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9841 - loss: 0.0511 - val_accuracy: 0.9794 - val_loss: 0.0646
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9876 - loss: 0.0395 - val_accuracy: 0.9793 - val_loss: 0.0674


通过命令行 （command） 或在 notebook 体验中启动 TensorBoard ，这两个接口通常是相同的。 在 notebooks, 使用 `%tensorboard` 命令。 在命令行中， 运行不带“％”的相同命令。

In [10]:
%tensorboard --logdir {temp_dir}/logs/fit

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_model_fit.png?raw=1"/> -->

在此示例中创建的可视化效果的简要概述以及可以找到它们的信息中心（顶部导航栏中的标签页）：

- **标量**显示损失和指标在每个周期如何变化。您还可以使用它们跟踪训练速度、学习率和其他标量值。可以在 **Time Series** 或 **Scalars** 信息中心找到标量。
- **计算图**可以帮助您呈现模型。在这种情况下，将显示层的 Keras 计算图，这可以帮助您确保正确构建。可以在 **Graphs** 信息中心找到计算图。
- **直方图**和**分布**显示张量随时间的分布。这对于呈现权重和偏差并验证它们是否以预期的方式变化很有用。可以在 **Time Series** 或 **Histograms** 信息中心中找到直方图。可以在 **Distributions** 信息中心中找到分布。

当您记录其他类型的数据时，会自动启用其他 TensorBoard 信息中心。 例如，使用 Keras TensorBoard 回调还可以记录图像和嵌入向量。您可以通过点击右上角的“inactive”下拉列表来查看 TensorBoard 中还有哪些其他信息中心。

## 通过其他方法使用 TensorBoard


用以下方法训练时，例如 [`tf.GradientTape()`](https://tensorflow.google.cn/api_docs/python/tf/GradientTape), 会使用 `tf.summary` 记录所需的信息。

使用与上述相同的数据集，但将其转换为 `tf.data.Dataset` 以利用批处理功能：

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

训练代码遵循 [advanced quickstart](https://tensorflow.google.cn/tutorials/quickstart/advanced) 教程，但显示了如何将 log 记录到 TensorBoard 。 首先选择损失和优化器：

In [12]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

创建可用于在训练期间累积值并在任何时候记录的有状态指标：

In [13]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

定义训练和测试代码：

In [14]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

设置摘要编写器，以将摘要写到另一个日志目录中的磁盘上：

In [18]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = temp_dir/'logs/gradient_tape' / current_time / 'train'
test_log_dir = temp_dir/'logs/gradient_tape' / current_time / 'test'
train_summary_writer = tf.summary.create_file_writer(str(train_log_dir))
test_summary_writer = tf.summary.create_file_writer(str(test_log_dir))

开始训练。使用 `tf.summary.scalar()` 在摘要编写器范围内的训练/测试期间记录指标（损失和准确率）以将摘要写入磁盘。您可以控制记录哪些指标以及记录的频率。其他 `tf.summary` 函数可以记录其他类型的数据。

In [19]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.24809525907039642, Accuracy: 92.788330078125, Test Loss: 0.11344233900308609, Test Accuracy: 96.73999786376953


AttributeError: 'Mean' object has no attribute 'reset_states'

再次打开 TensorBoard，这次将其指向新的日志目录。 我们也可以启动 TensorBoard 来监视训练进度。

In [ ]:
%tensorboard --logdir {temp_dir}/logs/gradient_tape

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_gradient_tape.png?raw=1"/> -->

您现在已经了解了如何通过 Keras 回调和通过 `tf.summary` 使用 TensorBoard 来实现更多自定义场景。 

## TensorBoard.dev：托管并共享您的机器学习实验结果

[TensorBoard.dev](https://tensorboard.dev) 是一项免费的公共服务，可让您上传您的 TensorBoard 日志并获得可在学术论文、博文、社交媒体等中与所有人共享的永久链接。这有助于实现更好的重现性和协作。

要使用 TensorBoard.dev，请运行以下命令：


In [ ]:
!tensorboard dev upload \
  --logdir {temp_dir}/logs/fit \
  --name "(optional) My latest experiment" \
  --description "(optional) Simple comparison of several hyperparameters" \
  --one_shot

请注意，此调用使用感叹号前缀 (`!`) 来调用 shell，而不是使用百分比前缀 (`%`) 来调用 colab 魔法。从命令行调用此命令时，不需要任何前缀。

在[此处](https://tensorboard.dev/experiment/EDZb7XgKSBKo6Gznh3i8hg/#scalars)查看示例。

要了解如何使用 TensorBoard.dev 的更多详细信息，请参阅 https://tensorboard.dev/#get-started